# Import Data Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [5]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

# Importing Data

In [2]:
path =  r'/Users/isomwinton/Desktop/Desktop - Isom’s MacBook Pro/Career Foundry/Data Immersion/Achievement 6/USA Real Estate Analysis 03-2024'

In [3]:
#import pickle file
usa = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'real_estate_cleaned_3-10-2024_revised2.pkl'))


In [45]:
# Import ".json" file for the U.S. 

country_geo = r'/Users/isomwinton/Desktop/Desktop - Isom’s MacBook Pro/Career Foundry/Data Immersion/Achievement 6/Exercise 6 practice/us-states.json'
                                         

In [4]:
usa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 946899 entries, 0 to 2001665
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   status          946899 non-null  category      
 1   bed             946899 non-null  float64       
 2   bath            946899 non-null  float64       
 3   acre_lot        946899 non-null  float64       
 4   city            946835 non-null  object        
 5   state           946899 non-null  object        
 6   zip_code        946899 non-null  object        
 7   house_size      946899 non-null  float64       
 8   prev_sold_date  576677 non-null  datetime64[ns]
 9   price           946899 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(5), object(3)
memory usage: 73.1+ MB


In [6]:
usa.shape

(946899, 10)

In [7]:
# Sampling 1% of the data
sampled_usa = usa.sample(frac=0.01, random_state=42)

In [8]:
# Apply the conditions to categorize prices of houses
sampled_usa.loc[(sampled_usa['price'] >= 10000) & (sampled_usa['price'] < 50000), 'price_category'] = 'Cheapest Housing'
sampled_usa.loc[(sampled_usa['price'] >= 50000) & (sampled_usa['price'] < 100000), 'price_category'] = 'Low-end Affordable'
sampled_usa.loc[(sampled_usa['price'] >= 100000) & (sampled_usa['price'] < 250000), 'price_category'] = 'Affordable'
sampled_usa.loc[(sampled_usa['price'] >= 250000) & (sampled_usa['price'] < 400000), 'price_category'] = 'Middle Affordable'
sampled_usa.loc[(sampled_usa['price'] >= 400000) & (sampled_usa['price'] < 650000), 'price_category'] = 'Upper Affordable'
sampled_usa.loc[(sampled_usa['price'] >= 650000) & (sampled_usa['price'] < 800000), 'price_category'] = 'High-End'
sampled_usa.loc[(sampled_usa['price'] > 800000), 'price_category'] = 'Luxury'


In [9]:
sampled_usa['price_category'].value_counts(dropna = False)

price_category
Affordable            2846
Middle Affordable     2197
Upper Affordable      1815
Luxury                1022
Low-end Affordable     745
High-End               661
Cheapest Housing       178
nan                      5
Name: count, dtype: int64

In [10]:
sampled_usa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9469 entries, 1573687 to 1327000
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   status          9469 non-null   category      
 1   bed             9469 non-null   float64       
 2   bath            9469 non-null   float64       
 3   acre_lot        9469 non-null   float64       
 4   city            9469 non-null   object        
 5   state           9469 non-null   object        
 6   zip_code        9469 non-null   object        
 7   house_size      9469 non-null   float64       
 8   prev_sold_date  5778 non-null   datetime64[ns]
 9   price           9469 non-null   float64       
 10  price_category  9469 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(5), object(4)
memory usage: 823.0+ KB


In [11]:
# change 'price_category' variable to a categorical data type because it contains a limited set of values
sampled_usa['price_category'] = sampled_usa['price_category'].astype('category')

In [12]:
sampled_usa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9469 entries, 1573687 to 1327000
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   status          9469 non-null   category      
 1   bed             9469 non-null   float64       
 2   bath            9469 non-null   float64       
 3   acre_lot        9469 non-null   float64       
 4   city            9469 non-null   object        
 5   state           9469 non-null   object        
 6   zip_code        9469 non-null   object        
 7   house_size      9469 non-null   float64       
 8   prev_sold_date  5778 non-null   datetime64[ns]
 9   price           9469 non-null   float64       
 10  price_category  9469 non-null   category      
dtypes: category(2), datetime64[ns](1), float64(5), object(3)
memory usage: 758.6+ KB


In [19]:
# value counts of the states to make sure there are no misspellings, etc.
state_counts = sampled_usa['state'].value_counts()

In [20]:
state_counts

state
New York          3225
Pennsylvania      2664
Massachusetts      943
New Jersey         728
Connecticut        703
New Hampshire      243
Rhode Island       214
Vermont            202
Maine              188
Puerto Rico        151
West Virginia       66
Ohio                64
Maryland            58
Delaware            14
Virginia             4
Virgin Islands       2
Name: count, dtype: int64

In [23]:
sampled_usa.columns

Index(['status', 'bed', 'bath', 'acre_lot', 'city', 'state', 'zip_code',
       'house_size', 'prev_sold_date', 'price', 'price_category'],
      dtype='object')

## Data Wrangling to Create Choropleth

In [21]:
# Create Dummy columns for States with only necessary df columns

columns = [
    "New York",
    "Pennsylvania",
    "Massachusetts",
    "New Jersey",
    "Connecticut",
    "New Hampshire",
    "Rhode Island",
    "Vermont",
    "Maine",
    "Puerto Rico",
    "West Virginia",
    "Ohio",
    "Maryland",
    "Delaware",
    "Virginia",
    "Virgin Islands",
    "status",
    "bath",
    "house_size",
    "price_category"
]


In [24]:
# Create a subset

state_rec = sampled_usa[columns]

KeyError: "['New York', 'Pennsylvania', 'Massachusetts', 'New Jersey', 'Connecticut', 'New Hampshire', 'Rhode Island', 'Vermont', 'Maine', 'Puerto Rico', 'West Virginia', 'Ohio', 'Maryland', 'Delaware', 'Virginia', 'Virgin Islands'] not in index"

In [49]:
# Not sure why I'm experiencing the error above so I will try another way by using pandas to create dummy columns for the state names then concatenate the original data frame back

state_dummies = pd.get_dummies(sampled_usa['state'])

In [50]:
# Create a new DataFrame that includes the columns 'status', 'bath', 'house_size', and 'price_category' from sampled_usa, along with all columns from state_dummies

usa_sample_dummies = pd.concat([
    state_dummies, 
    sampled_usa[['status', 'bath', 'house_size', 'price','price_category']]
], axis=1)


In [51]:
usa_sample_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9469 entries, 1573687 to 1327000
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Connecticut     9469 non-null   bool    
 1   Delaware        9469 non-null   bool    
 2   Maine           9469 non-null   bool    
 3   Maryland        9469 non-null   bool    
 4   Massachusetts   9469 non-null   bool    
 5   New Hampshire   9469 non-null   bool    
 6   New Jersey      9469 non-null   bool    
 7   New York        9469 non-null   bool    
 8   Ohio            9469 non-null   bool    
 9   Pennsylvania    9469 non-null   bool    
 10  Puerto Rico     9469 non-null   bool    
 11  Rhode Island    9469 non-null   bool    
 12  Vermont         9469 non-null   bool    
 13  Virgin Islands  9469 non-null   bool    
 14  Virginia        9469 non-null   bool    
 15  West Virginia   9469 non-null   bool    
 16  status          9469 non-null   category
 17  bath      

In [52]:
# Select only the states from state_rec in a new subset

states_only = state_dummies[["New York",
    "Pennsylvania",
    "Massachusetts",
    "New Jersey",
    "Connecticut",
    "New Hampshire",
    "Rhode Island",
    "Vermont",
    "Maine",
    "Puerto Rico",
    "West Virginia",
    "Ohio",
    "Maryland",
    "Delaware",
    "Virginia",
    "Virgin Islands"
]]

In [53]:
# This command turns the dummy data from the states columns into a categorical variable in s2, which is a pandas Series data structure

s2 = states_only.idxmax(axis=1)


In [54]:
s2

1573687     Pennsylvania
632390          Delaware
240524      Rhode Island
1864138     Pennsylvania
346553     Massachusetts
               ...      
242847      Rhode Island
463699       Connecticut
4789         Puerto Rico
1751646         New York
1327000         New York
Length: 9469, dtype: object

In [55]:
type(s2)

pandas.core.series.Series

In [56]:
# Create new column 'state_name' in the usa_sample_dummies dataframe

usa_sample_dummies['state_name'] = s2

In [57]:
usa_sample_dummies.columns

Index(['Connecticut', 'Delaware', 'Maine', 'Maryland', 'Massachusetts',
       'New Hampshire', 'New Jersey', 'New York', 'Ohio', 'Pennsylvania',
       'Puerto Rico', 'Rhode Island', 'Vermont', 'Virgin Islands', 'Virginia',
       'West Virginia', 'status', 'bath', 'house_size', 'price',
       'price_category', 'state_name'],
      dtype='object')

In [58]:
# Drop the dummy columns from the dataframe

usa_sample_dummies.drop(columns = ["New York",
    "Pennsylvania",
    "Massachusetts",
    "New Jersey",
    "Connecticut",
    "New Hampshire",
    "Rhode Island",
    "Vermont",
    "Maine",
    "Puerto Rico",
    "West Virginia",
    "Ohio",
    "Maryland",
    "Delaware",
    "Virginia",
    "Virgin Islands"], inplace = True)

In [59]:
usa_sample_dummies.columns

Index(['status', 'bath', 'house_size', 'price', 'price_category',
       'state_name'],
      dtype='object')

In [60]:
usa_sample_dummies.head()

status  bath  house_size     price     price_category  \
1573687  for_sale   2.0      2244.0  239900.0         Affordable   
632390   for_sale   4.0      4780.0  689900.0           High-End   
240524   for_sale   3.0      2130.0  599900.0   Upper Affordable   
1864138  for_sale   2.0      2092.0  299900.0  Middle Affordable   
346553   for_sale   1.0       960.0  240000.0         Affordable   

            state_name  
1573687   Pennsylvania  
632390        Delaware  
240524    Rhode Island  
1864138   Pennsylvania  
346553   Massachusetts

## Conduct Consistency Checks

In [61]:
# Check for missing values

usa_sample_dummies.isnull().sum()

status            0
bath              0
house_size        0
price             0
price_category    0
state_name        0
dtype: int64

### Check for duplicates

In [62]:
dups = usa_sample_dummies.duplicated()

In [63]:
dups.shape # No duplicates

(9469,)

## Plotting a Choropleth

In [64]:
# Create a data frame with just the states and the price we want plotted

data_to_plot = usa_sample_dummies[['state_name','price']]
data_to_plot.head()

state_name     price
1573687   Pennsylvania  239900.0
632390        Delaware  689900.0
240524    Rhode Island  599900.0
1864138   Pennsylvania  299900.0
346553   Massachusetts  240000.0

In [68]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [100, 0], zoom_start = 1.5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data = country_geo, 
    data = data_to_plot,
    columns = ['state_name', 'price'],
    key_on = 'feature.properties.name', # this part is very important - check your json file to see where the KEY is located
    fill_color = 'YlOrBr', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Price").add_to(map)
folium.LayerControl().add_to(map)

map

## 8. Our choropleth map shows the prices of homes concentrated in the Northeast of the United States. We can see New England states such as Vermont, New Hampshire, Massachusetts and Connecticut having the highest prices. I would suspect this is due to them being in close proximity to big cities like Boston, New York, D.C., etc. as are places where those working in the cities can get more space for their money instead of living in the cities. Also, these places are popular "second-home" places for high net worth individuals and familys to spend their summer and winter vacations. 

In [70]:
# Save the map as an HTML file
map.save('state_real_estate_interactive_map.html')